In [1]:
# computational libs
from scipy.spatial import Delaunay
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
from matplotlib import cm
import time
import sparse
import random
# %matplotlib

#vtk libs
from tvtk.api import tvtk
from mayavi.scripts import mayavi2
from PyQt5 import QtCore
import sip
import vtk

#emc3 libs
import numpy as np
import xarray as xr
import netCDF4

In [2]:
# importing the mayavi plotting lib
from mayavi import mlab
# this must be run as the first line, telling jupyter that can be interactively used
%gui qt

In [3]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [4]:
#from grid import grid

In [5]:
# importing widget libs to make interactive plots
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [6]:
# open the dataset containing the emc3 example
path = r"~/Dokumente/emc3_example.nc"
ds = xr.open_dataset(path)
#ds

In [7]:
def make_parammesh_vtk(grid_obj, sgrid, param_torgrid, ret_nans = False):
    """
    Args:np sparsenp sparse
     - grid_obj, grid object defined in one of the cells below
     - sgrid, tvtk.StructureGrid object
     - param_torgrid, type ndarray, parameter of interest on a toroidal meshgrid structure
     - renans, bool, return NaN vales of points found in a cell or not
    Out:
     - parammesh, ndarray, interpolated parameter values on a 3d cartesian grid
    """
    import numpy as np
    import vtk
    from tvtk.api import tvtk
    import itertools
    # make a dict to store potential nan values that appear in points found in a cell
    dictnans = {}
    notfound = []
    
    # make the mehsgrid for the parameter we want to interpolate form the original data
    parammesh = np.nan*np.ones(grid_obj.cart_x.shape)
    
    # pass initial/default values to the find_cell function
    cell = None
    cid = int(6e3)
    tol2 = 1e-3
    subid = vtk.reference(-1)
    pcoord = np.zeros((3))
    weights = np.zeros(8)
    
     #making coarse grid
    numc = 20
    coarse = np.zeros((numc, numc, numc), dtype=bool)
    spatial_toroidal = np.array([grid_obj.__dict__["tor_"+d] for d in "xyz"])
    minmax = [np.array([func(grid_obj.__dict__["tor_"+d]) for d in "xyz"]) for func in [np.min, np.max]]
    offset = minmax[0]
    dist = (minmax[1] - minmax[0]+1e-6)/numc
    for i,j,k in itertools.product(*[range(x-1) for x in grid_obj.tor_x.shape]):
        block = spatial_toroidal[:,i:i+2, j:j+2, k:k+2].reshape(3, 8)
        minmax = [func(block, axis=1) for func in [np.min, np.max]]
        indices = [((m - offset) / dist).astype(int) for m in minmax]
        for ijk in itertools.product(*[range(a,b+1) for a,b in zip(*indices)]):
            coarse[ijk] = True
    
    # iterate through points in the cartesian grid
    for pid, p in enumerate(grid_obj.cartflat):
        # check the coarse grid for points first
        if not coarse[tuple(((p-offset)/dist).astype(int) )]:
            continue
        #find the cell p is interior to
        cid = sgrid.find_cell(p, cell, cid, tol2, subid, pcoord, weights)
        # if find_cell returns -1 there is either a numerical error or it did not find the point in that cell
        if cid == -1:
            notfound.append(cid)
            continue
        else:            
            # unravel index of each vertex of the cell found from the toroidal grid
            newgridshape = (grid_obj.tor_x.shape[0]-1, grid_obj.tor_x.shape[1]-1, grid_obj.tor_x.shape[2]-1)

            # get indicies of the cell corner
            ir, it, ip = np.unravel_index(cid, newgridshape)

            #find the potential values of each vertex of the cell and interpolate the data using weights
            pvals = param_torgrid[ir:(ir+2), it:(it+2), ip:(ip+2)].ravel() * weights
            paramval = np.sum(pvals, axis = 0)

            # find coordinate indices of the point in the cartesian grid structure with cartesian indicies
            coord_idx = np.argwhere((grid_obj.cart_x == p[0]) & 
                                    (grid_obj.cart_y == p[1]) & 
                                    (grid_obj.cart_z == p[2]))[0] 
            # Assign the cartesian parameter mesh the parameter value
            parammesh[coord_idx[0], coord_idx[1], coord_idx[2]] = paramval
            
    # if user input chosen return nans then return nans 
    if ret_nans:
        # generate different return when including the dictionary containing the nans
        return parammesh, dictnans, notfound
    else:
        return parammesh, notfound

In [8]:
%%writefile grid.py

class grid:
    def __init__(self, ds, ngrid_cart = None, inc_r = [None, None], inc_t = [None, None], inc_p = [None, None]):
        """
        Args:
         - xarray, ds is a dataset containing the W7X data
         - np.int64, ngrid_cart is cartesian grid resolution 
         - list dim 2 (min,max), inc_r, inc_t, inc_p is the minmax of the grid indexes in r, theta and phi direction
         
        Out:
         - grid object, both cartesian meshgrid of toroidal and cartesian grid, can also get as flattened arrays
        
        Note: Periodic boundary conditions is added in theta direction
        """
        import numpy as np
        
        dgrid = np.asarray( ds.R_bounds.mean(dim=("delta_r", "delta_theta", "delta_phi")) )[inc_r[0]:inc_r[1],
                                                                                            inc_t[0]:inc_t[1],
                                                                                            inc_p[0]:inc_p[1]].shape
        
        self.R, self.theta, self.phi, self.tor_z = np.ones(dgrid)*np.nan, np.ones(dgrid)*np.nan, np.ones(dgrid)*np.nan, np.ones(dgrid)*np.nan
        
        self.R = np.asarray( ds.R_bounds.mean(dim=("delta_r", "delta_theta", "delta_phi")) )[inc_r[0]:inc_r[1],
                                                                                             inc_t[0]:inc_t[1],
                                                                                             inc_p[0]:inc_p[1]]

        
        self.tor_z = np.asarray(ds.z_bounds.mean(dim=("delta_r", "delta_theta", "delta_phi")))[inc_r[0]:inc_r[1],
                                                                                               inc_t[0]:inc_t[1],
                                                                                               inc_p[0]:inc_p[1]]

        
        self.phi = np.asarray( ds.phi_bounds.mean(dim="delta_phi") )[inc_p[0]:inc_p[1]]

        
        self.tor_x = np.asarray( self.R * np.cos(self.phi) )
        self.tor_y = np.asarray( self.R * np.sin(self.phi) )
        if ngrid_cart == None:
            ngrid_cart = 100
        
        ivl_p = int(inc_p[1] - inc_p[0])
        chunking = 2
        chunkingids = (2,3,4)
        self.cart_x, self.cart_y, self.cart_z = np.meshgrid(np.linspace(np.min(self.tor_x),np.max(self.tor_x), ngrid_cart), 
                                                            np.linspace(np.min(self.tor_y),np.max(self.tor_y), ngrid_cart), 
                                                            np.linspace(np.min(self.tor_z),np.max(self.tor_z), ngrid_cart),
                                                            indexing = "ij")
        """self.cart_x, self.cart_y, self.cart_z = np.meshgrid(np.linspace(np.min(self.tor_x),np.max(self.tor_x), ngrid_cart), 
                                                            np.linspace(np.min(self.tor_y),np.max(self.tor_y), ngrid_cart), 
                                                            np.linspace(np.min(self.tor_z),np.max(self.tor_z), ngrid_cart),
                                                            indexing = "ij")"""
     
        self.cartflat = np.ones((self.cart_x.shape[0]*self.cart_x.shape[1]*self.cart_x.shape[2], 3) )*np.nan
        self.cartflat[:,0] = self.cart_x.flatten()
        self.cartflat[:,1] = self.cart_y.flatten()
        self.cartflat[:,2] = self.cart_z.flatten()
        
        self.torflat = np.ones((self.tor_x.shape[0]*self.tor_x.shape[1]*self.tor_x.shape[2], 3))*np.nan
        self.torflat[:,0] = self.tor_x.flatten()
        self.torflat[:,1] = self.tor_y.flatten()
        self.torflat[:,2] = self.tor_z.flatten()

Overwriting grid.py


In [9]:
%%writefile makemesh_indices.py
def make_parammesh_vtk_indices(grid_obj, sgrid, skip_coarsegrid = 10, sp = (29,2,16), tolsqrd = 1e-8):
    """
    Args:
     - grid_obj, grid object defined in one of the cells below
     - sgrid, tvtk.StructureGrid object
     - renans, bool, return NaN vales of points found in a cell or not
    Out:
     - parammesh_indices, ndarray np.int32, interpolated parameter vertex indices and weights on a 3d cartesian grid
     dim = (x, y, z, coord index vertices of cell/weights of vertices)
    """
    import numpy as np
    import vtk
    from tvtk.api import tvtk
    import itertools
    # need to have grid.py in the same folder as this py file
    from grid import grid
    
    # make the mehsgrid for the parameter we want to interpolate form the original data
    # + (2,8) represents (indices/weights, vertex)
    parammesh_indices = -np.ones(grid_obj.cart_x.shape + (8,), dtype=np.int32 )
    parammesh_weights = np.nan*np.ones(grid_obj.cart_x.shape + (8,), dtype=np.float32)
    # pass initial/default values to the find_cell function
    cell = None
    cid = int(6e3)
    tol2 = tolsqrd
    print(tol2)
    subid = vtk.reference(-1)
    pcoord = np.zeros(3)
    weights = np.zeros(8, dtype=np.float32)
    
    # generate the shape of the cellular emc3 grid
    grid_of_cells_shape = (grid_obj.tor_x.shape[0]-1, grid_obj.tor_x.shape[1]-1, grid_obj.tor_x.shape[2]-1)
    
    # for storing cell ids of the emc3 grid to each grid point in the cartesian grid
    cids = np.zeros(grid_obj.cart_x.shape, dtype=np.int32)-2
    cs = sp # this is now hardcoded, need to find a generalized solution
    todo = [tuple(list(cs))] #starting index of point to search for in the cells to search
    found = False
    while todo:
        cur = todo.pop()
        #find the cell p is interior to
        cid = sgrid.find_cell([grid_obj.cart_x[cur], grid_obj.cart_y[cur], grid_obj.cart_z[cur]], 
                              cell, cid, tol2, subid, pcoord, weights)
        cids[cur] = cid
        if cid >= 0:
            found = True
            for dist in itertools.product(range(-1,2),repeat=3): #iterate through neighbours
                nxt = tuple(a+b for a,b in zip(dist, cur))
                outofbounds = False
                for i in range(3):
                    if nxt[i]>=grid_obj.cart_x.shape[i]:
                        outofbounds = True
                if outofbounds:
                    continue
                if cids[nxt] == -2:
                    cids[nxt] = -3
                    todo.append(nxt)
            # unravel index of each vertex of the cell found from the toroidal grid
            ir, it, ip = np.unravel_index(cid, grid_of_cells_shape)
            idx = np.zeros((2,2,2), dtype=int)
            for d_ir, dit, dip in itertools.product(range(2), repeat=3):
                idx[d_ir, dit, dip] = np.ravel_multi_index((ir+d_ir,it+dit,ip+dip), grid_obj.tor_x.shape)
            idx.shape = 8
            #represents the indices of the flattened parametermesh on the emc3 grid structure
#             print(parammesh_indices.shape, parammesh_indices.dtype, cur, type(cur), idx, idx.dtype, weights, weights.dtype)
            parammesh_indices[cur] = idx
            # represents the weights corresponding to the parameter indices above
            parammesh_weights[cur] = weights
        #if find_cell returns -1 there is either a numerical error or it did not find the point in that cell
        if not found:
            print("guessing startpoint")
            cs = tuple(np.random.randint(low = 0, high = max(grid_obj.cart_x.shape), size = 3))
            todo.append(cs)
    return parammesh_indices, parammesh_weights

def from_indices_to_paramvals(idxs, weights, param_torgrid):
    """
    Current problem is that it is not identical to the former method where the the parametervalues are extracted
    rather than the indices of the toroidal grid.
    """
    #imports for importing to external file later
    import numpy as np
    # shape pmesh as (50,50,50), if ngrid = 50
    pmesh = np.ones(idxs.shape[:3], dtype=np.float64)*np.nan
    # make a flattened array of sum of indices of all vertices in the cell, if any is nan then discard the cell
    irav = np.sum(idxs, axis = 3).ravel()
    # store indices of indices which are numbers in a flattened structjupyter labextension install @jupyter-widgets/jupyterlab-managerure
    isnotnan = np.where(np.isnan(irav) == False)[0]
    # array of indexes that is not nanvalued
    notnanidxs = np.asarray(idxs.reshape(idxs.shape[0]**3,8)[isnotnan], dtype=np.int32)
    pmesh_re = pmesh.ravel()
    pmesh_re[isnotnan] = np.sum(param_torgrid.ravel()[notnanidxs] * weights.reshape(weights.shape[0]**3, 8)[isnotnan], axis = 1)
    pmesh = pmesh_re.reshape(idxs.shape[:3])
    return pmesh   

Overwriting makemesh_indices.py


In [10]:
from grid import grid
grid_emc3 = grid(ds = ds, ngrid_cart = 128, inc_r = [40, 138], inc_t = [0,512], inc_p = [0,10])

In [11]:
dimgrid = list([x for x in grid_emc3.tor_x.shape[::-1]])
sgrid_emc3 = tvtk.StructuredGrid(dimensions=dimgrid)
sgrid_emc3.points = grid_emc3.torflat

sgridvtk = vtk.vtkStructuredGrid()
sgridvtk.points = grid_emc3.torflat 

"""usgrid_emc3 = tvtk.UnstructuredGrid(points = grid_emc3.torflat)
tet_type = tvtk.Hexahedron().cell_type # VTK_TETRA == 10
hexs = np.arange(len(grid_emc3.torflat), len(grid_emc3.torflat)-1)
usgrid_emc3.set_cells(tet_type, hexs)"""

'usgrid_emc3 = tvtk.UnstructuredGrid(points = grid_emc3.torflat)\ntet_type = tvtk.Hexahedron().cell_type # VTK_TETRA == 10\nhexs = np.arange(len(grid_emc3.torflat), len(grid_emc3.torflat)-1)\nusgrid_emc3.set_cells(tet_type, hexs)'

In [12]:

"""Plotting the actual values in the W7X grid.
"""

"""# View the data.
@mayavi2.standalone
def view():
    from mayavi.sources.vtk_data_source import VTKDataSource
    from mayavi.modules.api import Outline, GridPlane, StructuredGridOutline

    mayavi.new_scene()
    src = VTKDataSource(data=sgrid_emc3)
    mayavi.add_source(src)
    mayavi.add_module(Outline())
    g = GridPlane()
    g.grid_plane.axis = 'x'
    mayavi.add_module(g)
    g = GridPlane()
    g.grid_plane.axis = 'y'
    mayavi.add_module(g)
    g = GridPlane()
    g.grid_plane.axis = 'z'
    mayavi.add_module(g)

if __name__ == '__main__':
    view()"""

"# View the data.\n@mayavi2.standalone\ndef view():\n    from mayavi.sources.vtk_data_source import VTKDataSource\n    from mayavi.modules.api import Outline, GridPlane, StructuredGridOutline\n\n    mayavi.new_scene()\n    src = VTKDataSource(data=sgrid_emc3)\n    mayavi.add_source(src)\n    mayavi.add_module(Outline())\n    g = GridPlane()\n    g.grid_plane.axis = 'x'\n    mayavi.add_module(g)\n    g = GridPlane()\n    g.grid_plane.axis = 'y'\n    mayavi.add_module(g)\n    g = GridPlane()\n    g.grid_plane.axis = 'z'\n    mayavi.add_module(g)\n\nif __name__ == '__main__':\n    view()"

In [13]:
from makemesh_indices import make_parammesh_vtk_indices,from_indices_to_paramvals
tik = time.time()
idxmesh, weightmesh = make_parammesh_vtk_indices(grid_obj = grid_emc3,
                                                 sgrid = sgrid_emc3, sp = (0, 0, 0))
tok = time.time()
runtime_idxmesh = tok-tik
print("To make indicies it took ", runtime_idxmesh)

1e-08
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
guessing startpoint
To make indicies it took  57.42478919029236


In [14]:
tik = time.time()
idxmesh_hightol, weightmesh_hightol = make_parammesh_vtk_indices(grid_obj = grid_emc3,
                                                                 sgrid = sgrid_emc3, sp = (0, 0, 0), 
                                                                 tolsqrd = 2e-2)
tok = time.time()
runtime_idxmesh = tok-tik
print("To make indicies it took ", runtime_idxmesh)

0.02
guessing startpoint
To make indicies it took  80.81538391113281


In [15]:
tik = time.time()
pmeshtest = from_indices_to_paramvals(idxs = idxmesh, 
                                      weights = weightmesh, 
                                      param_torgrid = np.asarray(ds.Te)[40:138,:512,:10])
tok = time.time()
print("To evaluate indicies took ", tok - tik)

To evaluate indicies took  0.41826534271240234


In [18]:
tik = time.time()
pmeshtest_hightol = from_indices_to_paramvals(idxs = idxmesh_hightol,
                                              weights = weightmesh_hightol,
                                              param_torgrid = np.asarray(ds.Te)[40:138,:512,:10])
tok = time.time()
print("To evaluate indicies took ", tok - tik)

To evaluate indicies took  0.3087947368621826


In [30]:
%matplotlib
def plot_Te(iy):
    plt.figure(figsize=(10,10))
    im = plt.imshow(np.sum([pmeshtest_hightol[:,iy].T[::-1],-pmeshtest[:,iy].T[::-1]], axis=0))
    plt.colorbar(im)

iy = widgets.IntSlider(min = 0, max = pmeshtest.shape[1]-1, value = 20)
interact(plot_Te, iy = iy)

<function __main__.plot_Te(iy)>

In [ ]:
"""
Comparing the interpolated region with what I should see.
"""
def plot_emc3toroidal(ip):
    figemc3 = plt.figure(figsize=(10,10))
    axemc3 = figemc3.add_subplot()
    clev = np.arange(ds.Te[40:138,:,ip].min(),ds.Te[40:138,:,ip].max(),.5)
    cemc3 = axemc3.contourf(grid_emc3.R[:,:,ip], 
                            grid_emc3.tor_z[:,:,ip], 
                            ds.Te[40:138,:,ip], 
                            clev)
    figemc3.colorbar(cemc3)
phislider = widgets.IntSlider(min = 0, max = 3, value = 0)
interact(plot_emc3toroidal, ip = phislider)

In [ ]:
print(pmeshtest[64:69,4,78:84])
pmeshtest[65,4,80]

In [ ]:
cell = None
cid = int(6e3)
tol2 = 1e-8
subid = vtk.reference(-1)
pcoord = np.zeros(3)
weights = np.zeros(8, dtype=np.float32)
out = np.zeros((100,100), dtype=int)
for i in range(100):
    for j in range(100):
        out[j, i] = sgrid_emc3.find_cell([grid_emc3.cart_x[65,4,80], 
                                  grid_emc3.cart_y[65,4,80]-(i-50)/1280/2, 
                                  grid_emc3.cart_z[65,4,80]-(j-50)/1280/2],
                                 cell, cid, tol2, subid, pcoord, weights)
    #print(i, found)
plt.figure(figsize=(10,10))
plt.imshow(out, origin='lower')
plt.colorbar()
#plt.figure(figsize=(10,10))
#%matplotlib qt
#print(out[:,18])
#print(out[1:,18]- out[:-1,18])
#plt.plot(out[:,18])
#-312571 + 307972, -307972   +    0   +    0 + 303373

In [ ]:
hexahedron_emc3 = sgrid_emc3.get_cell(sgrid_emc3.find_cell([grid_emc3.cart_x[105,4,45], 
                                                            grid_emc3.cart_y[105,4,45], 
                                                            grid_emc3.cart_z[105,4,45]],
                                                           cell, cid, tol2, subid, pcoord, weights))

def main():
    colors = vtk.vtkNamedColors()

    # Set the background color.
    colors.SetColor("BkgColor", [51, 77, 102, 255])

    # For the hexahedron setup the coordinates of eight points.
    # The two faces must be in counter clockwise order as viewed from the
    # outside.
    pointCoordinates = hexahedron_emc3.points.to_array()
    # Create the points.
    points = vtk.vtkPoints()

    # Create a hexahedron from the points.
    hexahedron = vtk.vtkHexahedron()

    for i in range(0, len(pointCoordinates)):
        points.InsertNextPoint(pointCoordinates[i])
        hexahedron.GetPointIds().SetId(i, i)

    # Add the hexahedron to a cell array.
    hexs = vtk.vtkCellArray()
    hexs.InsertNextCell(hexahedron)

    # Add the points and hexahedron to an unstructured grid.
    uGrid = vtk.vtkUnstructuredGrid()
    uGrid.SetPoints(points)
    uGrid.InsertNextCell(hexahedron.GetCellType(), hexahedron.GetPointIds())

    # Visualize.
    mapper = vtk.vtkDataSetMapper()
    mapper.SetInputData(uGrid)

    actor = vtk.vtkActor()
    actor.GetProperty().SetColor(colors.GetColor3d("PeachPuff"))
    actor.SetMapper(mapper)

    renderer = vtk.vtkRenderer()
    renderWindow = vtk.vtkRenderWindow()
    renderWindow.SetWindowName("Hexahedron")
    renderWindow.AddRenderer(renderer)
    renderWindowInteractor = vtk.vtkRenderWindowInteractor()
    renderWindowInteractor.SetRenderWindow(renderWindow)

    renderer.AddActor(actor)
    renderer.SetBackground(colors.GetColor3d("BkgColor"))
    renderer.ResetCamera()
    renderer.GetActiveCamera().Azimuth(30)
    renderer.GetActiveCamera().Elevation(30)

    renderWindow.Render()
    renderWindowInteractor.Start()


if __name__ == '__main__':
    main()

In [ ]:
from matplotlib import pyplot as plt
hexahedron_emc3 = sgrid_emc3.get_cell(307972)
points3d = hexahedron_emc3.points.to_array()
points3d.shape
inv = points3d[::-1]
mlab.clf()
mlab.points3d(points3d[:,0], points3d[:,1], points3d[:,2], scale_factor=1.5e-3)
mlab.axes()
mlab.outline()

In [ ]:
unrav = np.unravel_index(307972, grid_emc3.tor_x.shape)
unrav

In [ ]:
import xemc3
idx=65,4,80
xyz = [grid_emc3.__dict__["cart_"+d][idx] for d in "xyz"]
x,y,z =xyz
r = np.sqrt(x**2+y**2)
phi = np.arctan2(y,x)
ds["tmp"] = ds.Te.copy()
ds.tmp.data[...] = 0

plt.figure(figsize=(20,20))
ds.tmp.data[60,77] = 1
ds.emc3.plot_Rz("tmp", phi)
plt.scatter(r, z)
#ds.tmp.data.shape
unrav